<a href="https://www.kaggle.com/code/mdajwadhossain/sentiment-analysis-on-yt-comments-with-knn-lr-nb?scriptVersionId=225113648" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Import necessary libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
import re

In [2]:
# Step 1: Load the dataset (already cleaned it)
df = pd.read_csv('/kaggle/input/youtube-comments-dataset/YoutubeCommentsDataSet.csv')

df.head()

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,neutral
1,here in nz 50 of retailers don’t even have con...,negative
2,i will forever acknowledge this channel with t...,positive
3,whenever i go to a place that doesn’t take app...,negative
4,apple pay is so convenient secure and easy to ...,positive


In [3]:
df = df.dropna(subset=['Comment'])

In [4]:
# Standardize Column Names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

In [5]:
print(df.columns)

Index(['comment', 'sentiment'], dtype='object')


In [6]:
# Replace 'neg' with 0, 'pos' with 1, and 'nue' with 2
df['sentiment'] = df['sentiment'].replace({'negative': 0, 'neutral': 1, 'positive': 2})

df.head()

<ipython-input-6-32ecfc6ec5bc>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'negative': 0, 'neutral': 1, 'positive': 2})


,comment,sentiment
0,lets not forget that apple pay in 2014 require...,1
1,here in nz 50 of retailers don’t even have con...,0
2,i will forever acknowledge this channel with t...,2
3,whenever i go to a place that doesn’t take app...,0
4,apple pay is so convenient secure and easy to ...,2


In [7]:
 # Clean Text (Remove Special Characters, Links, etc.)

def clean_text(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r'http\S+', '', text)  # Remove links
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'#\w+', '', text)  # Remove hashtags
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    return text.strip()
# Example usage
df['comment'] = df['comment'].apply(clean_text)
df.head()

,comment,sentiment
0,lets not forget that apple pay in required a ...,1
1,here in nz of retailers dont even have contac...,0
2,i will forever acknowledge this channel with t...,2
3,whenever i go to a place that doesnt take appl...,0
4,apple pay is so convenient secure and easy to ...,2


In [8]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Example usage
df['comment'] = df['comment'].apply(remove_stopwords)
df.head()

,comment,sentiment
0,lets forget apple pay required brand new iphon...,1
1,nz retailers dont even contactless credit card...,0
2,forever acknowledge channel help lessons ideas...,2
3,whenever go place doesnt take apple pay doesnt...,0
4,apple pay convenient secure easy use used kore...,2


In [9]:
# Step 2: Split the data into features (X) and target (y)
X = df['comment']  
y = df['sentiment']

In [10]:
# Step 3: Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Step 4: Convert text data into numerical features using TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000) 
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [12]:
# Step 5: Initialize models
models = {
    "Logistic Regression": LogisticRegression(multi_class='ovr', max_iter=4000),

    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=1000), # You can change n_neighbors as needed

    "Naive Bayes": MultinomialNB(alpha=1),


}

In [13]:
# Step 6: Train and evaluate each model
for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    model.fit(X_train_tfidf, y_train)
    
    # Predict the sentiments for the test set
    y_pred = model.predict(X_test_tfidf)
    
    # Evaluate the model
    print(f"Accuracy Score for {model_name}: {accuracy_score(y_test, y_pred)}")
    print(f"Classification Report for {model_name}:\n{classification_report(y_test, y_pred)}")


Training Logistic Regression...
Accuracy Score for Logistic Regression: 0.7487067791995644
Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.65      0.26      0.37       441
           1       0.63      0.55      0.58       912
           2       0.79      0.92      0.85      2320

    accuracy                           0.75      3673
   macro avg       0.69      0.58      0.60      3673
weighted avg       0.73      0.75      0.73      3673


Training K-Nearest Neighbors...
Accuracy Score for K-Nearest Neighbors: 0.6536890824938743
Classification Report for K-Nearest Neighbors:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       441
           1       0.69      0.14      0.23       912
           2       0.65      0.98      0.78      2320

    accuracy                           0.65      3673
   macro avg       0.45      0.37      0.34      3673
weighted avg   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
